In [49]:
import csv
from re import I, X
from scipy.linalg import eigh
import numpy as np
import matplotlib.pyplot as plt

In [50]:
def load_data(filepath):
    
    with open(filepath, encoding="UTF8") as csvfile:
        pokemon = csv.DictReader(csvfile)

        output = []
        for data in pokemon:
            output.append(data)
            
        # pop #,Name,Type 1,Type 2,Total
        popData = ['#', 'Name', 'Type 1', 'Type 2', 'Total', 'Generation', 'Legendary']
        for data in output:
            for key in popData:
                data.pop(key)
    return output

In [51]:
# load_data check
pokemon_list = load_data("Pokemon.csv")
# display(pokemon_list)

In [66]:
# input: dict representing one Pokemon
# output: numpy array of shape (6,) and  dtype int64. The first element is x1 and so
# on with the sixth element being x6.
def calc_features(row):
    output = np.dtype(np.int64)
    output = np.array(list(row.items()))
    return output

In [67]:
# calc_features check
np_pokemon_list = []
np_pokemon_list.append(calc_features(pokemon_list[0]))
np_pokemon_list.append(calc_features(pokemon_list[1]))
np_pokemon_list.append(calc_features(pokemon_list[2]))
np_pokemon_list.append(calc_features(pokemon_list[3]))
np_pokemon_list.append(calc_features(pokemon_list[4]))
np_pokemon_list.append(calc_features(pokemon_list[5]))
display(np_pokemon_list)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [ ]:
def hac(features):
    
    pass

In [ ]:
# hac check
hac(np_pokemon_list)

In [ ]:
def imshow_hac(Z):
    return plt.show();

In [ ]:
# Final check
hac([calc_features(row) for row in load_data('Pokemon.csv')][:6])

In [ ]:
linkage([calc_features(row) for row in load_data('Pokemon.csv')][:6], method='complete')

In [ ]:
# Check plot
imshow_hac(linkage([calc_features(row) for row in load_data('Pokemon.csv')][:10], method='complete'))